In [1]:
import requests
import json
import pandas as pd
import numpy as np
import ift6758
from pathlib import Path

In [2]:
import os
from pathlib import Path
import logging
from flask import Flask, jsonify, request, abort, g
import xgboost
import requests
import pandas as pd
from comet_ml import API
import joblib

import sklearn
import ift6758

In [3]:
try:
    hey = requests.get("http://0.0.0.0:5000/hello")
except Exception as e:
    print(e)

In [4]:
hey.text

'Hello!'

In [22]:
logs = requests.get("http://0.0.0.0:5000/logs")
logs.json()[-5:]

['    validate_features=validate_features,',
 '  File "/usr/local/lib/python3.7/site-packages/xgboost/core.py", line 2129, in inplace_predict',
 '    f"Feature shape mismatch, expected: {self.num_features()}, "',
 'ValueError: Feature shape mismatch, expected: 29, got 27',
 '2022-12-18 17:47:10,061;INFO;Accessed page /logs']

In [6]:
default_model = {
    'workspace': 'ift-6758-projet-quipe-13',
    'model': 'xgboost-best-all-features',
    'version': '1.0.0'
}

In [7]:
model = requests.post("http://0.0.0.0:5000/download_registry_model", json=default_model)

In [8]:
model.text

'"Updated from local folder."\n'

In [9]:
df = pd.DataFrame(np.random.randn(5266, 29))

In [10]:
pred = requests.post("http://0.0.0.0:5000/predict", json=df.to_json(orient='split'))

In [11]:
xd = pd.read_json(pred.json(), orient='split')

In [12]:
xd

,0,1
0,0.603409,0.396591
1,0.703120,0.296880
2,0.678572,0.321428
3,0.792988,0.207012
4,0.788031,0.211969
...,...,...
5261,0.741485,0.258515
5262,0.734705,0.265295
5263,0.799895,0.200105
5264,0.675646,0.324354


In [15]:
from ift6758.data import pd_make_df

In [16]:
from ift6758.client import game_client

In [17]:
gc = game_client.GameClient()

In [18]:
df = gc.process_query(2022020329)

In [23]:
df.columns

Index(['periodTimeSec', 'period', 'coordinate_x', 'coordinate_y',
       'distanceFromGoal', 'shotAngle', 'lastEventCoord_y', 'timeDifference',
       'distanceDifference', 'rebound', 'shotAngleDifference', 'speed',
       'Backhand', 'Deflected', 'Slap Shot', 'Snap Shot', 'Tip-In',
       'Wrist Shot', 'BLOCKED_SHOT', 'FACEOFF', 'GIVEAWAY', 'HIT',
       'MISSED_SHOT', 'SHOT', 'TAKEAWAY', 'Wrap-around', 'GOAL'],
      dtype='object')

In [19]:
pred = requests.post("http://0.0.0.0:5000/predict", json=df.to_json(orient='split'))

In [21]:
pred.text

'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'